In [1]:
%pylab inline
import numpy as np
import pandas as pd
from pymongo import MongoClient
from tqdm import tqdm
import hashlib

import config

ImportError: No module named pymongo

In [63]:
def _img_from_bytes(img_bytes):
    np_img_str = np.fromstring(img_bytes, np.uint8)
    img = cv2.imdecode(np_img_str, cv2.IMREAD_COLOR)
    assert img is not None
    return img

In [ ]:
def get_md5(img):
    return hashlib.md5(img).hexdigest()

In [ ]:
train = dict()
counts = dict()

client = MongoClient(connect=False)
db = client.cdiscount['train']
for product in tqdm(db.find(), total=config.PRODUCT_COUNT):
    assert 1 <= len(product['imgs']) <= 4
    for i in range(len(product['imgs'])):
            # res.append((product['_id'], i, product['category_id']))
            hash_ = get_md5(product['imgs'][i]['picture'])
            train[hash_] = (product['_id'], i, product['category_id'])
            if hash_ not in counts:
                counts[hash_] = 1
            else:
                counts[hash_] += 1
path = 'train_hash.pickle'
with open(path, 'wb') as f:
    pickle.dump(train, f)
with open('counts.pickle', 'wb') as f:
    pickle.dump(counts, f)
print('Done.')

In [2]:
listed = counts.items()
listed.sort(key=lambda x : x[1], reversed=True)
print(listed[:10])

In [60]:
for hash_, count in listed[:10]:
    product_id, img_num, cat = train[hash_]
    product_id = int(product_id)
    image_number = int(image_number)
    product = db.find_one({'_id': product_id})
    assert product is not None
    img = _img_from_bytes(product['imgs'][image_number]['picture'])
    plt.imshow(img)
    plt.show()

{1: 2, 3: 5}

In [61]:
z = x.items()
z

[(1, 2), (3, 5)]

In [62]:
z.sort(key=lambda x : x[1], reverse=True)
z

[(3, 5), (1, 2)]